In [22]:
import os
import numpy as np
import pandas as pd
import csv
# This line tells the notebook to show plots inside of the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sb

In [23]:
os.chdir('D:\\ice_data\\train_clean')
ice_data_15=pd.read_csv('train_cleandata_15.csv',sep=',')
ice_data_21=pd.read_csv('train_cleandata_21.csv',sep=',')

In [24]:
ice_data_15.shape
# ice_data_21.shape

(374147, 28)

In [25]:
ice_data_15.head()

,label,wind_speed,generator_speed,power,wind_direction,wind_direction_mean,yaw_position,yaw_speed,pitch1_angle,pitch2_angle,...,acc_y,environment_tmp,int_tmp,pitch1_ng5_tmp,pitch2_ng5_tmp,pitch3_ng5_tmp,pitch1_ng5_DC,pitch2_ng5_DC,pitch3_ng5_DC,group
0,0,1.859993,1.223595,2.515790,-2.072739,-2.073627,-0.655343,0.030804,0.555556,0.506667,...,0.061109,-0.403919,0.014918,1.307692,1.123077,0.783077,1.36,0.00,1.56,1
1,0,1.911625,1.293394,2.313551,-2.010591,-1.615140,-0.655343,0.030804,0.195556,0.133333,...,-1.209522,-0.421277,-0.002291,1.307692,1.123077,0.783077,0.44,2.88,-2.60,1
2,0,1.635027,1.280099,2.507799,-2.053750,-0.282742,-0.649566,0.170338,0.964444,0.951111,...,0.061109,-0.421277,-0.002291,1.307692,1.123077,0.783077,1.76,0.60,2.56,1
3,0,1.786234,1.280099,2.349593,-2.007138,-2.234477,-0.655343,-0.004080,0.168889,0.137778,...,0.061109,-0.403919,-0.002291,1.307692,1.123077,0.783077,2.80,-0.48,0.12,1
4,0,1.786234,1.263480,2.321566,-2.264365,-1.428959,-0.637917,0.414524,0.182222,0.168889,...,-1.209522,-0.403919,0.014918,1.307692,1.123077,0.783077,-0.88,1.72,0.92,1


In [26]:
label_15=ice_data_15.label
ice_data_15 = ice_data_15.drop('label',axis=1)
ice_data_15.head()

,wind_speed,generator_speed,power,wind_direction,wind_direction_mean,yaw_position,yaw_speed,pitch1_angle,pitch2_angle,pitch3_angle,...,acc_y,environment_tmp,int_tmp,pitch1_ng5_tmp,pitch2_ng5_tmp,pitch3_ng5_tmp,pitch1_ng5_DC,pitch2_ng5_DC,pitch3_ng5_DC,group
0,1.859993,1.223595,2.515790,-2.072739,-2.073627,-0.655343,0.030804,0.555556,0.506667,0.551111,...,0.061109,-0.403919,0.014918,1.307692,1.123077,0.783077,1.36,0.00,1.56,1
1,1.911625,1.293394,2.313551,-2.010591,-1.615140,-0.655343,0.030804,0.195556,0.133333,0.191111,...,-1.209522,-0.421277,-0.002291,1.307692,1.123077,0.783077,0.44,2.88,-2.60,1
2,1.635027,1.280099,2.507799,-2.053750,-0.282742,-0.649566,0.170338,0.964444,0.951111,0.960000,...,0.061109,-0.421277,-0.002291,1.307692,1.123077,0.783077,1.76,0.60,2.56,1
3,1.786234,1.280099,2.349593,-2.007138,-2.234477,-0.655343,-0.004080,0.168889,0.137778,0.177778,...,0.061109,-0.403919,-0.002291,1.307692,1.123077,0.783077,2.80,-0.48,0.12,1
4,1.786234,1.263480,2.321566,-2.264365,-1.428959,-0.637917,0.414524,0.182222,0.168889,0.204444,...,-1.209522,-0.403919,0.014918,1.307692,1.123077,0.783077,-0.88,1.72,0.92,1


In [27]:
type(list(label_15))

list

In [28]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import copy
from sklearn.preprocessing import MinMaxScaler
dt = DecisionTreeClassifier()

基于决策树的模型  
每次删除一个特征
使用ROC进行评价  
由于数据还是有一定的尺寸差别，为了减少这个差别，我们对他进行缩放

In [29]:
X_train_o=copy.deepcopy(ice_data_15)
y_train_o=label_15

In [30]:
from sklearn.model_selection import train_test_split
# 随机选择百分之10的数据
X_train, X_test1, y_train, y_test1 = train_test_split(X_train_o, y_train_o, test_size=0.1, random_state=128, shuffle = True)# shuffle默认为True
# 在选择的数据中，选择2/3作为训练集，1/3作为测试集
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=128, shuffle = False)# shuffle默认为True


In [31]:
# 归一化
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_scaled = min_max_scaler.fit_transform(X_train)
X_test_scaled = min_max_scaler.fit_transform(X_test)



#准备工作到此结束，此后的代码均不改变X_train, X_test1, y_train, y_test1 ，X_train_scaled等变量的值

In [32]:
from sklearn import svm
from sklearn.metrics import classification_report
regr = svm.SVC(kernel='rbf')
regr.fit(X_train_scaled, y_train)
y_pre_svc=regr.predict(X_test_scaled)
print(classification_report(y_true=y_test, y_pred=y_pre_svc))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99     94624
           1       0.92      0.63      0.75      6396

    accuracy                           0.97    101020
   macro avg       0.95      0.81      0.87    101020
weighted avg       0.97      0.97      0.97    101020



In [33]:
label_21=ice_data_21.label
ice_data_21 = ice_data_21.drop('label',axis=1)
X_test_scaled_21 = min_max_scaler.fit_transform(ice_data_21)
y_pre_21=SDGclf.predict(X_test_scaled_21)
print(classification_report(y_true=label_21, y_pred=y_pre_21))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97    168930
           1       0.00      0.00      0.00     10638

    accuracy                           0.94    179568
   macro avg       0.47      0.50      0.48    179568
weighted avg       0.89      0.94      0.91    179568



In [53]:
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_sample(X_train_scaled, y_train)
SDGclf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
SDGclf.fit(X_resampled, y_resampled)
y_pre=SDGclf.predict(X_test_scaled)
print(classification_report(y_true=y_test, y_pred=y_pre))

              precision    recall  f1-score   support

           0       0.99      0.87      0.93     94624
           1       0.31      0.84      0.45      6396

    accuracy                           0.87    101020
   macro avg       0.65      0.85      0.69    101020
weighted avg       0.94      0.87      0.90    101020



In [18]:
from sklearn.linear_model import SGDClassifier
y_pre=[]
SDGclf = SGDClassifier(loss="hinge", penalty="l2", max_iter=1000)
SDGclf.fit(X_train_scaled, y_train)
y_pre=SDGclf.predict(X_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pre))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     94624
           1       0.98      0.14      0.25      6396

    accuracy                           0.95    101020
   macro avg       0.96      0.57      0.61    101020
weighted avg       0.95      0.95      0.93    101020



In [21]:
label_21=ice_data_21.label
ice_data_21 = ice_data_21.drop('label',axis=1)
X_test_scaled_21 = min_max_scaler.fit_transform(ice_data_21)
y_pre_21=SDGclf.predict(X_test_scaled_21)
print(classification_report(y_true=label_21, y_pred=y_pre_21))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97    168930
           1       0.00      0.00      0.00     10638

    accuracy                           0.94    179568
   macro avg       0.47      0.50      0.48    179568
weighted avg       0.89      0.94      0.91    179568



###这个代码运行时间较长
###运行得到决策树的分类结果及不同评价指标特征个数-性能图
from sklearn.tree import DecisionTreeClassifier
y_pre=[]
dt = DecisionTreeClassifier()
dt.fit(X_train_scaled, y_train)
y_pre=dt.predict(X_test_scaled)
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test, y_pred=y_pre))
rfecv = RFECV(estimator=dt, step=2, cv=StratifiedKFold(2), scoring='roc_auc')
rfecv.fit(X_train_o, y_train_o)
print("Optimal number of features : %d" % rfecv.n_features_)
print("Ranking of features names: %s" % X_train_o.columns[rfecv.ranking_-1])
print("Ranking of features nums: %s" % rfecv.ranking_)
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.savefig("feature.png")
plt.show()

In [39]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(max_depth=146,n_estimators=2500, 
                                    max_leaf_nodes=2500, oob_score=True, random_state=30, n_jobs=-1)
rf_clf.fit(X_train, y_train)
y_predict = rf_clf.predict(X_test)
print(rf_clf.oob_score_)

KeyboardInterrupt: 

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False,
                         title='Confusion matrix',
                         cmap=plt.cm.Blues):
    """
    Normalization can be applied by setting `normalize = True`.
    """
    plt.figure()
    plt.imshow(cm, interpolation='nearest',cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:,np.newaxis]
        print("Normalized confusion matrix")
    else:
        print("Confusion matrix, without normalization")
    print(cm)
    
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j, i, cm[i,j],
                horizontalalignment="center",
                color="white" if cm[i,j] > thresh else "black")
        
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig("matrix.png")
  

In [ ]:
rom sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

prediction = rf_clf.predict(X_test1)
cm = confusion_matrix(y_test1, prediction)
cm_plot_labels = ['normal', 'failure']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')


In [ ]:
#precision & recall & f1-score
from sklearn.metrics import classification_report
print(classification_report(y_true=y_test1, y_pred=prediction))